In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import scipy.io as scio
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import time
import matplotlib
import matplotlib.pyplot as plt
# from models import *
# from collections import OrderedDict

In [ ]:
import matplotlib.patches as patches

In [ ]:
# Folders
TrainingDataFolder = "/datadisk/sdc/wangruibo/share1/QWL_DataBase/SuperRes_ChanEst/SISO_1x1/Diversity_LocationVelocity/210421_TrainData_NLOS1Square/"
TestingDataFolder  = "/datadisk/sdc/wangruibo/share1/QWL_DataBase/SuperRes_ChanEst/SISO_1x1/Diversity_LocationVelocity/210421_TestData_NLOS1Square/"
ThisFileName       = "SuperResChanEst-RCAN-20200427-(8NlosABCD)" 
Which_new = 'NET_E'
ModuleFolder       = "/datadisk/sdc/wangruibo/Location_new_way/" + Which_new

In [ ]:
class dataset(Dataset):
    def __init__(self, input_path, target_path, mode = 'train', seedNN = 0):
        super(dataset, self).__init__()
        self.input_data = []
        self.target_data = []
        for i_path in input_path:
            i_data = scio.loadmat(i_path)['tmpData']
            self.input_data.append(i_data)
        for t_path in target_path:
            self.target_data.append(scio.loadmat(t_path)['tmpData'])
        self.input_data = np.concatenate(self.input_data, axis=0)
        self.target_data = np.concatenate(self.target_data, axis=0)
        self.length = self.target_data.shape[0]
        
        # 随机打乱顺序
        random.seed(seedNN)
        ind_shuffle = random.sample(range(0, self.length), self.length)

        if mode == 'train':
            self.target_data = self.target_data[ind_shuffle]
            self.input_data = self.input_data[ind_shuffle]
        else:
            self.target_data = self.target_data[ind_shuffle]
            self.input_data = self.input_data[ind_shuffle]
        self.length = self.target_data.shape[0]
#         print(self.input_data.shape)
#         print(self.target_data.shape)
        
    def __getitem__(self, index):
        input_data = torch.from_numpy(self.input_data[index])[:, :, 0].unsqueeze(2)
        target_data = torch.from_numpy(self.target_data[index])
        return input_data, target_data
        
    def __len__(self):
        return self.length

In [ ]:
# 41 Locations see data description
'''
for splitted data loader
'''
def DataSplitter(LocPatchIndex, LocationList, VelocityList, seedNN = 0):
    input_path11 = [TrainingDataFolder + 'Loc ' + LocPatchIndex + '/RawCe_PilotOnly/10ReDmrs_Ant1_RxPos{}_RxVel{}_ModOrd1_RawCe_PilotOnly-1.mat'.format(i, j) for i in LocationList for j in VelocityList]
    target_path1 = [TrainingDataFolder + 'Loc ' + LocPatchIndex + '/Rx_PerfectCeDataPilotComb_fxt/10ReDmrs_Ant1_RxPos{}_RxVel{}_ModOrd1_Rx_PerfectCeDataPilotComb_fxt-1.mat'.format(i,j) for i in LocationList for j in VelocityList]
    
    input_path21 = [TrainingDataFolder + 'Loc ' + LocPatchIndex + '/RawCe_PilotOnly Plus/10ReDmrs_Ant1_RxPos{}_RxVel{}_ModOrd1_RawCe_PilotOnly-1.mat'.format(i, j) for i in LocationList for j in VelocityList]
    target_path2 = [TrainingDataFolder + 'Loc ' + LocPatchIndex + '/Rx_PerfectCeDataPilotComb_fxt Plus/10ReDmrs_Ant1_RxPos{}_RxVel{}_ModOrd1_Rx_PerfectCeDataPilotComb_fxt-1.mat'.format(i,j) for i in LocationList for j in VelocityList]
    
#     trainset    = dataset(input_path11, target_path1, mode='train', seedNN = seedNN)
#     trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

#     testset    = dataset(input_path21, target_path2, mode='test', seedNN = seedNN)
#     testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)
    
    return input_path11, target_path1,input_path21,target_path2

In [ ]:
LocPatchList = ['A']
LocationList = range(1, 442) 
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
trainset    = dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

testset    = dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
# testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)

In [ ]:
class one_point_dataset(Dataset):
    def __init__(self, input_path, target_path, mode = 'train', seedNN = 0):
        super(one_point_dataset, self).__init__()
        self.input_data = []
        self.target_data = []
        for i_path in input_path:
            i_data = scio.loadmat(i_path)['tmpData']
            self.input_data.append(i_data)
        for t_path in target_path:
            self.target_data.append(scio.loadmat(t_path)['tmpData'])
        self.input_data = np.concatenate(self.input_data, axis=0)
        self.target_data = np.concatenate(self.target_data, axis=0)
        self.length = self.target_data.shape[0]
        
        # 随机打乱顺序
        random.seed(seedNN)
        ind_shuffle = random.sample(range(0, self.length), self.length)

        if mode == 'train':
            self.target_data = self.target_data
            self.input_data = self.input_data
#                 self.target_data = np.expand_dims(self.target_data, axis=0)
#                 self.input_data = np.expand_dims(self.input_data, axis=0)
        else:
            self.target_data = self.target_data[ind_shuffle]
            self.input_data = self.input_data[ind_shuffle]
        self.length = self.target_data.shape[0]
#         print(self.input_data.shape)
#         print(self.target_data.shape)
        
    def __getitem__(self, index):
        input_data = torch.from_numpy(self.input_data[index])[:, :, 0].unsqueeze(2)
        target_data = torch.from_numpy(self.target_data[index])
        return input_data, target_data
        
    def __len__(self):
        return self.length

In [ ]:
LocPatchList = ['A']
LocationList = list(range(1,6)) + list(range(22,27)) + list(range(43,48)) + list(range(64,69))
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
# testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)

In [ ]:
trainloader=DataLoader(one_trainset, batch_size=64, shuffle=True, num_workers=8)
testloader=DataLoader(one_testset, batch_size=64, shuffle=True, num_workers=8)

/datadisk/sdc/wangruibo/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
"""
Network structure from high to low:
RCAN -- RG -- RCAB --ChannelAttention

"""
class ChannelAttention(nn.Module):
    def __init__(self, num_features, reduction):
        super(ChannelAttention, self).__init__()
        self.module = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(num_features, num_features // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features // reduction, num_features, kernel_size=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return x * self.module(x)
    

class RCAB(nn.Module):
    def __init__(self, num_features, reduction):
        super(RCAB, self).__init__()
        self.module = nn.Sequential(
            nn.Conv2d(num_features, num_features, kernel_size=(3,1), padding=(1,0)),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features, num_features, kernel_size=(3,1), padding=(1,0)),
            ChannelAttention(num_features, reduction)
        )
        
    def forward(self, x):
        return x + self.module(x)
    
    
class RG(nn.Module):
    def __init__(self, num_features, num_rcab, reduction):
        super(RG, self).__init__()
        self.module = [RCAB(num_features, reduction) for _ in range(num_rcab)]
        self.module.append(nn.Conv2d(num_features, num_features, kernel_size=(3,1),padding=(1,0)))
        self.module = nn.Sequential(*self.module)
        
    def forward(self, x):
        return x + self.module(x)
    
# 2*22*1->2*238*11
'''
class RCAN(nn.Module):
    def __init__(self, num_features, num_rg, num_rcab, reduction):
        super(RCAN, self).__init__()
        # 64*2*22*1
        self.sf = nn.Conv2d(2, num_features, kernel_size=(3,1), padding=(1,0))
        # 64*32*22*1
        self.rgs = nn.Sequential(*[RG(num_features, num_rcab, reduction) for _ in range(num_rg)])
        # 64*32*22*1
        self.conv1 = nn.Conv2d(num_features, num_features, kernel_size=3, padding=1)
        # 64*32*22*1
        self.Tran = nn.Conv2d(num_features, num_features, kernel_size=3, padding=(0,1))
        # 64*32*20*1
        self.Trelu = nn.ReLU()
        self.upscale1 = nn.Conv2d(num_features, num_features*4, kernel_size=3, padding=1)
        # reshape之后变为64*32*40*2
        self.relu1 = nn.ReLU()
        self.upscale2 = nn.Conv2d(num_features, num_features*6, kernel_size=3, padding=1)
        # reshape之后变为64*32*80*6
        self.relu2 = nn.ReLU()
        self.upscale3 = nn.Conv2d(num_features, num_features*6, kernel_size=3, padding=1)
        # reshape之后变为64*32*240*12
        self.relu3 = nn.ReLU()
        self.conv2 = nn.Conv2d(num_features, 2, kernel_size=(3,2), padding=0)
        # 64*2*238*11
        # self.relu4 = nn.ReLU()
        # self.conv3 = nn.Conv2d(num_features, 2, kernel_size=3, padding=1)
        self.num_features = num_features
        
    def forward(self, x):
        x = self.sf(x)
        residual = x
        x = self.rgs(x)
        x = self.conv1(x)
        x += residual
        x = self.Trelu(self.Tran(x))
        x = self.upscale1(x)
        # 类似于nn.PixelShuffle
        x = x.view(x.shape[0], self.num_features, 40, 2)
        x = self.relu1(x)
        x = self.upscale2(x)
        x = x.view(x.shape[0], self.num_features, 80, 6)
        x = self.relu2(x)
        x = self.upscale3(x)
        x = x.view(x.shape[0], self.num_features, 240, 12)
        x = self.relu3(x)
        x = self.conv2(x)
        # x = self.relu4(x)
        # x = self.conv3(x)
        return x
'''


# 2*22*1->2*22*50
class RCAN(nn.Module):
    def __init__(self, num_features, num_rg, num_rcab, reduction):
        super(RCAN, self).__init__()
        # 64*2*22*1
        self.sf = nn.Conv2d(2, num_features, kernel_size=(3,1), padding=(1,0))
        # 64*32*22*1
        self.rgs = nn.Sequential(*[RG(num_features, num_rcab, reduction) for _ in range(num_rg)])
        # 64*32*22*1
        self.conv1 = nn.Conv2d(num_features, num_features, kernel_size=3, padding=1)
        # 64*32*22*1
        self.Tran = nn.Conv2d(num_features, num_features, kernel_size=3, padding=(0,1))
        # 64*32*20*1
        self.Trelu = nn.ReLU()
        self.upscale1 = nn.Conv2d(num_features, num_features*6, kernel_size=3, padding=1)
        # reshape之后变为64*32*60*2
        self.relu1 = nn.ReLU()
        self.upscale2 = nn.Conv2d(num_features, num_features*10, kernel_size=3, padding=1)
        # reshape之后变为64*32*120*10
        self.relu2 = nn.ReLU()
        self.upscale3 = nn.Conv2d(num_features, num_features*10, kernel_size=3, padding=1)
        # reshape之后变为64*32*240*50
        self.relu3 = nn.ReLU()
        self.conv2 = nn.Conv2d(num_features, 2, kernel_size=3, padding=(0,1))
        # 64*2*238*50
        # self.relu4 = nn.ReLU()
        # self.conv3 = nn.Conv2d(num_features, 2, kernel_size=3, padding=1)
        self.num_features = num_features
        
    def forward(self, x):
        x = self.sf(x)
        residual = x
        x = self.rgs(x)
        x = self.conv1(x)
        x += residual
        x = self.Trelu(self.Tran(x))
        x = self.upscale1(x)
        # 类似于nn.PixelShuffle
        x = x.view(x.shape[0], self.num_features, 60, 2)
        x = self.relu1(x)
        x = self.upscale2(x)
        x = x.view(x.shape[0], self.num_features, 120, 10)
        x = self.relu2(x)
        x = self.upscale3(x)
        x = x.view(x.shape[0], self.num_features, 240, 50)
        x = self.relu3(x)
        x = self.conv2(x)
        # x = self.relu4(x)
        # x = self.conv3(x)
        return x

In [ ]:
def train(Epoch,load,path):
    net = RCAN(32, 3, 6, 8).cuda()
    if load:
        net.load_state_dict(torch.load(path))
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in net.parameters()))
    print(NetworkParamNum)
    
    lr = 0.001
    optimizer = optim.Adam(net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    min_testloss = 10000
    
    testError  = []
    trainError = []
    costTime   = []
    storeStepResult = True
    
    
    for epoch in range(Epoch):
        net.train()
        train_loss = 0.0
        test_loss = 0.
        time_start = time.time()
        numPatchList = len(LocPatchList)
#         for LocPathIndex in LocPatchList:
#             train_loader,test_loader = DataSplitter(LocPathIndex, LocationList, VelocityList)
            
        for dmrs, target in trainloader:
                optimizer.zero_grad()
                target = target.cuda().float()
                dmrs = dmrs.cuda().float()
                t_pred = net(dmrs)
                loss = torch.mean((t_pred - target) ** 2)
                loss.backward()
                optimizer.step()
                train_loss += float(loss.item())
        scheduler.step()
        numtr = len(trainloader)
            
        del t_pred, target, dmrs
        
        net.eval()
            
        for dmrs, target in testloader:
                target = target.cuda().float()
                dmrs = dmrs.cuda().float()
                t_pred = net(dmrs)
                loss = torch.mean((t_pred - target) ** 2)
                test_loss += loss.item()
        numte = len(testloader)
            
        del t_pred, target, dmrs
        
        train_loss = train_loss / numtr / numPatchList
        test_loss = test_loss / numte / numPatchList
        
        time_end = time.time()
        cost = time_end - time_start
        showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss, cost)
        
        print(showStrings, end='\n')
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        
        file = open(ModuleFolder +'/' +Which_new + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.close()
        
        torch.save(net.state_dict(), ModuleFolder + '/' + Which_new +'-Epk{}.pth'.format(epoch+1))

   

In [ ]:
Which_new = 'NET_A_small_area'
ModuleFolder       = "/datadisk/sdc/wangruibo/Location_new_way/" + Which_new
train(100,False,path = None)

Total number of paramerters in networks is 386058  
[Epoch:   1/100][TraingLoss: 0.19746][TestingLoss:0.05499][CostTime: 11.49206 s]
[Epoch:   2/100][TraingLoss: 0.01748][TestingLoss:0.00842][CostTime: 11.92161 s]
[Epoch:   3/100][TraingLoss: 0.00744][TestingLoss:0.00664][CostTime: 11.07932 s]
[Epoch:   4/100][TraingLoss: 0.00596][TestingLoss:0.00611][CostTime: 11.64112 s]
[Epoch:   5/100][TraingLoss: 0.00502][TestingLoss:0.00463][CostTime: 11.37527 s]
[Epoch:   6/100][TraingLoss: 0.00465][TestingLoss:0.00414][CostTime: 11.85929 s]
[Epoch:   7/100][TraingLoss: 0.00418][TestingLoss:0.00371][CostTime: 11.85567 s]
[Epoch:   8/100][TraingLoss: 0.00388][TestingLoss:0.00370][CostTime: 11.71499 s]
[Epoch:   9/100][TraingLoss: 0.00361][TestingLoss:0.00368][CostTime: 12.08047 s]
[Epoch:  10/100][TraingLoss: 0.00362][TestingLoss:0.00407][CostTime: 11.87859 s]
[Epoch:  11/100][TraingLoss: 0.00340][TestingLoss:0.00349][CostTime: 11.97583 s]
[Epoch:  12/100][TraingLoss: 0.00320][TestingLoss:0.00336

In [ ]:
del trainloader,testloader

In [ ]:
LocPatchList = ['A']
LocationList = list(range(169,174)) + list(range(190,195))+ list(range(148,153)) + list(range(127,132))
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
# testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
trainloader=DataLoader(one_trainset, batch_size=64, shuffle=True, num_workers=8)
testloader=DataLoader(one_testset, batch_size=64, shuffle=True, num_workers=8)
Which_new = 'NET_C_small_area'
ModuleFolder       = "/datadisk/sdc/wangruibo/Location_new_way/" + Which_new
train(100,False,path = None)

Total number of paramerters in networks is 386058  
[Epoch:   1/100][TraingLoss: 0.36102][TestingLoss:0.15364][CostTime: 12.44611 s]
[Epoch:   2/100][TraingLoss: 0.04788][TestingLoss:0.01474][CostTime: 13.04643 s]
[Epoch:   3/100][TraingLoss: 0.01226][TestingLoss:0.01004][CostTime: 13.41255 s]
[Epoch:   4/100][TraingLoss: 0.01052][TestingLoss:0.00872][CostTime: 12.83706 s]
[Epoch:   5/100][TraingLoss: 0.00764][TestingLoss:0.00788][CostTime: 12.56126 s]
[Epoch:   6/100][TraingLoss: 0.00699][TestingLoss:0.00618][CostTime: 12.37501 s]
[Epoch:   7/100][TraingLoss: 0.00611][TestingLoss:0.00632][CostTime: 12.33757 s]
[Epoch:   8/100][TraingLoss: 0.00563][TestingLoss:0.00530][CostTime: 12.18526 s]
[Epoch:   9/100][TraingLoss: 0.00535][TestingLoss:0.00533][CostTime: 12.77736 s]
[Epoch:  10/100][TraingLoss: 0.00505][TestingLoss:0.00648][CostTime: 12.23726 s]
[Epoch:  11/100][TraingLoss: 0.00478][TestingLoss:0.00415][CostTime: 12.59911 s]
[Epoch:  12/100][TraingLoss: 0.00432][TestingLoss:0.00453

In [ ]:
del trainloader,testloader

In [ ]:
LocPatchList = ['A']
LocationList = list(range(6,11)) + list(range(27,32)) + list(range(48,53)) + list(range(71,74))
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
# testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
trainloader=DataLoader(one_trainset, batch_size=64, shuffle=True, num_workers=8)
testloader=DataLoader(one_testset, batch_size=64, shuffle=True, num_workers=8)
Which_new = 'NET_B_small_area'
ModuleFolder       = "/datadisk/sdc/wangruibo/Location_new_way/" + Which_new
train(100,False,path = None)

Total number of paramerters in networks is 386058  
[Epoch:   1/100][TraingLoss: 0.10609][TestingLoss:0.02077][CostTime: 12.58017 s]
[Epoch:   2/100][TraingLoss: 0.00858][TestingLoss:0.00614][CostTime: 11.53275 s]
[Epoch:   3/100][TraingLoss: 0.00548][TestingLoss:0.00496][CostTime: 12.50478 s]
[Epoch:   4/100][TraingLoss: 0.00454][TestingLoss:0.00444][CostTime: 12.38915 s]
[Epoch:   5/100][TraingLoss: 0.00399][TestingLoss:0.00386][CostTime: 12.64756 s]
[Epoch:   6/100][TraingLoss: 0.00376][TestingLoss:0.00362][CostTime: 12.48521 s]
[Epoch:   7/100][TraingLoss: 0.00355][TestingLoss:0.00386][CostTime: 12.19822 s]
[Epoch:   8/100][TraingLoss: 0.00351][TestingLoss:0.00350][CostTime: 12.46903 s]
[Epoch:   9/100][TraingLoss: 0.00329][TestingLoss:0.00341][CostTime: 12.23566 s]
[Epoch:  10/100][TraingLoss: 0.00314][TestingLoss:0.00348][CostTime: 11.69034 s]
[Epoch:  11/100][TraingLoss: 0.00299][TestingLoss:0.00309][CostTime: 12.43343 s]
[Epoch:  12/100][TraingLoss: 0.00295][TestingLoss:0.00292

In [ ]:
del trainloader,testloader

In [ ]:
LocPatchList = ['A']
LocationList = list(range(186,190)) + list(range(196,200)) + list(range(143,148)) + list(range(169,169))
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
# testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
trainloader=DataLoader(one_trainset, batch_size=64, shuffle=True, num_workers=8)
testloader=DataLoader(one_testset, batch_size=64, shuffle=True, num_workers=8)
Which_new = 'NET_D_small_area'
ModuleFolder       = "/datadisk/sdc/wangruibo/Location_new_way/" + Which_new
train(100,False,path = None)

Total number of paramerters in networks is 386058  
[Epoch:   1/100][TraingLoss: 0.17435][TestingLoss:0.07729][CostTime: 11.30133 s]
[Epoch:   2/100][TraingLoss: 0.03948][TestingLoss:0.01509][CostTime: 11.23442 s]
[Epoch:   3/100][TraingLoss: 0.01163][TestingLoss:0.00998][CostTime: 11.39240 s]
[Epoch:   4/100][TraingLoss: 0.00915][TestingLoss:0.00830][CostTime: 11.48756 s]
[Epoch:   5/100][TraingLoss: 0.00796][TestingLoss:0.00753][CostTime: 10.86127 s]
[Epoch:   6/100][TraingLoss: 0.00755][TestingLoss:0.00687][CostTime: 11.19295 s]
[Epoch:   7/100][TraingLoss: 0.00669][TestingLoss:0.00640][CostTime: 11.72673 s]
[Epoch:   8/100][TraingLoss: 0.00608][TestingLoss:0.00635][CostTime: 11.68029 s]
[Epoch:   9/100][TraingLoss: 0.00584][TestingLoss:0.00574][CostTime: 11.11769 s]
[Epoch:  10/100][TraingLoss: 0.00559][TestingLoss:0.00561][CostTime: 11.42260 s]
[Epoch:  11/100][TraingLoss: 0.00516][TestingLoss:0.00578][CostTime: 11.54769 s]
[Epoch:  12/100][TraingLoss: 0.00502][TestingLoss:0.00474

In [ ]:
del trainloader,testloader

In [ ]:
net_A = RCAN(32, 3, 6, 8).cuda()
net_A.load_state_dict(torch.load('/datadisk/sdc/wangruibo/Location_new_way/NET_A_small_area/NET_A_small_area-Epk100.pth'))

<All keys matched successfully>

In [ ]:
net_B = RCAN(32, 3, 6, 8).cuda()
net_B.load_state_dict(torch.load('/datadisk/sdc/wangruibo/Location_new_way/NET_B_small_area/NET_B_small_area-Epk100.pth'))

<All keys matched successfully>

In [ ]:
net_C = RCAN(32, 3, 6, 8).cuda()
net_C.load_state_dict(torch.load('/datadisk/sdc/wangruibo/Location_new_way/NET_C_small_area/NET_C_small_area-Epk100.pth'))

<All keys matched successfully>

In [ ]:
net_D = RCAN(32, 3, 6, 8).cuda()
net_D.load_state_dict(torch.load('/datadisk/sdc/wangruibo/Location_new_way/NET_D_small_area/NET_D_small_area-Epk100.pth'))

<All keys matched successfully>

In [ ]:
LocPatchList = ['E']
LocationList = range(1, 442) 
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
trainset    = dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8, drop_last=False)

testset    = dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8, drop_last=False)

In [ ]:
'''
Basic Classes for building the network
'''
# For building Main Sub Branch

class ChannelAttention(nn.Module):
    def __init__(self, num_features, reduction):
        super(ChannelAttention, self).__init__()
        self.module = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(num_features, num_features // reduction, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features // reduction, num_features, kernel_size=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return x * self.module(x)

    
class RCAB(nn.Module):
    def __init__(self, num_features, reduction, k_size=(3,3), p=(1,1)):
        super(RCAB, self).__init__()
        self.module = nn.Sequential(
            nn.Conv2d(num_features, num_features, kernel_size=k_size, padding=p),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_features, num_features, kernel_size=k_size, padding=p),
            ChannelAttention(num_features, reduction)
        )
        
    def forward(self, x):
        return x + self.module(x)
    
    
class RG(nn.Module):
    def __init__(self, num_features, num_rcab, reduction, k_size=(3,3), p=(1,1)):
        super(RG, self).__init__()
        self.module = [RCAB(num_features, reduction, k_size, p) for _ in range(num_rcab)]
        self.module.append(nn.Conv2d(num_features, num_features, kernel_size=k_size,padding=p))
        self.module = nn.Sequential(*self.module)
        
    def forward(self, x):
        return x + self.module(x)
    
    
# 2*22*1->2*238*50
# 两个子网络的结果相乘
class PilotCAN(nn.Module):
    def __init__(self, num_features, num_rg, num_rcab, reduction):
        super(PilotCAN, self).__init__()
        
        # 2*156*1  --> 32*156*1
        self.sf = nn.Conv2d(2, num_features, kernel_size=(3,1), padding=(1,0))
        
        # 32*156*1 --> 32*156*1
        self.rgs = nn.Sequential(*[RG(num_features, num_rcab, reduction, k_size=(3,1), p=(1,0)) for _ in range(num_rg)])
        
        # 32*156*1 --> 32*156*1
        self.conv1 = nn.Conv2d(num_features, num_features, kernel_size=3, padding=1)
        
        # 32*156*1 --> 32*156*1
        self.Tran = nn.Conv2d(num_features, num_features, kernel_size=3, padding=1) 
        self.Trelu = nn.ReLU()
        
        # 32*156*1 --> 64*156*1 reshape --> 32*312*1
        self.upscale1 = nn.Conv2d(num_features, num_features*2, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        
        # 32*312*1--> 64*312*1 reshape --> 32*312*2
        self.upscale2 = nn.Conv2d(num_features, num_features*2, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        
        # 32*312*2--> 224*312*2 reshape --> 32*312*14
        self.upscale3 = nn.Conv2d(num_features, num_features*7, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        
        # 32*312*14 --> 2*312*14
        self.conv2 = nn.Conv2d(num_features, 2, kernel_size=3, padding=1)
        
        self.num_features = num_features
        
    def forward(self, x):
        
        # 2*156*1  --> 32*156*1
        x = self.sf(x)
        residual = x
        
        # 32*156*1 --> 32*156*1
        x = self.rgs(x)
        
        # 32*156*1 --> 32*156*1
        x = self.conv1(x)
        x += residual
        
        # 32*156*1 --> 32*156*1
        x = self.Tran(x)
        x = self.Trelu(x)
        
        # 32*156*1 --> 64*156*1 reshape --> 32*312*1
        x = self.upscale1(x)
        x = x.view(x.shape[0], self.num_features, x.shape[2]*2, 1)
        x = self.relu1(x)

        # 32*312*1--> 64*312*1 reshape --> 32*312*2
        x = self.upscale2(x)
        x = x.view(x.shape[0], self.num_features, 312, 2)
        x = self.relu2(x)

        # 32*312*2--> 224*312*2 reshape --> 32*312*14
        x = self.upscale3(x)
        x = x.view(x.shape[0], self.num_features, 312, 14)
        x = self.relu3(x)

        # 32*312*14 --> 2*312*14
        x = self.conv2(x)

        return x

In [ ]:
'''
Basic Classes for building the network
'''

# For building SAN related

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=3):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
    
# 带空间注意力的resnet模块
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, kernel_size=3, stride=1, padding=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=kernel_size, stride=1, padding=padding)
        self.bn2 = nn.BatchNorm2d(planes)
        self.sa = SpatialAttention()
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.sa(out) * out
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out
    
class ResNet_SAN(nn.Module):

    def __init__(self, layers=[2,2,2], num_feature=100):
        self.inplanes = 32
        super(ResNet_SAN, self).__init__()
        
        # B*1*201*201 --> B*32*201*201
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        
        # B*32*201*201 --> B*32*101*101
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # B*32*101*101 --> B*32*51*51
        self.layer1 = self._make_layer(32, layers[0], stride=2)
        
        # B*32*51*51 --> B*64*26*26
        self.layer2 = self._make_layer(64, layers[1], stride=2)
        
        # B*64*26*26 -> B*28*13*13
        self.layer3 = self._make_layer(4, layers[2], stride=2)
        
        # B*28*13*13 --> B*28*1*1
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(128*1, num_feature)

    def _make_layer(self, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * 1:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * 1,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * 1),
            )

        layers = []
        layers.append(BasicBlock(self.inplanes, planes, kernel_size=3, stride=stride, 
                                 padding=1, downsample=downsample))
        self.inplanes = planes * 1
        for i in range(1, blocks):
            layers.append(BasicBlock(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        
        # B*1*201*201 --> B*32*201*201
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        # B*32*201*201 --> B*32*101*101
        x = self.maxpool(x)

        # B*32*101*101 --> B*32*51*51
        x = self.layer1(x)
        
        # B*32*51*51 --> B*64*26*26
        x = self.layer2(x)
        
        # B*64*26*26--> B*28*13*13
        x = self.layer3(x)
        
        # B*28*13*13--> B*28*1*1
        x = self.avgpool(x)
        
        x = x.view(x.size(0), -1).squeeze()
        return x

In [ ]:
pattern_net = ResNet_SAN().cuda()

In [ ]:
pattern_net(torch.rand(1,2,22,1).cuda()).shape

torch.Size([4])

In [ ]:
def train_pr(Epoch,pr_net,train_loader,test_loader,*net):
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in pr_net.parameters()))
    print(NetworkParamNum)
#     model_list = [i for i in net]
    lr = 0.001
    optimizer = optim.Adam(pr_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    min_testloss = 10000
    
    testError  = []
    trainError = []
    costTime   = []
    storeStepResult = True
    
    
    for epoch in range(Epoch):
        pr_net.train()
        train_loss = 0.0
        test_loss = 0.0
        time_start = time.time()
        for input_x, label in trainloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
            with torch.no_grad():
                    for i in net:
                        i.eval()
                        result_net_list.append(i(input_x))
            result_net_list = torch.cat(result_net_list)
            result_mul_add = []
            for i in range(4,result_net_list.shape[0]+1,4):
                result_mul_add.append(torch.sum(result_net_list[:i],dim = 0,keepdim= True))
            result_mul_add = torch.cat(result_mul_add)
            
            optimizer.zero_grad()
            t_pred = pr_net(input_x)
            result_add = []    
            for i,j in zip(t_pred,result_mul_add):
                result_temp = []
                for m in i:
                    result_temp.append(m * result_mul_add)
                result_temp = torch.cat(result_temp)
                result_add.append(torch.sum(result_temp,dim=0,keepdim = True))
            result_add = torch.cat(result_add)
            result_add = torch.nn.functional.normalize(result_add,dim = 1)
            loss = torch.mean((result_add - label) ** 2)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
        scheduler.step()
        numtr = len(trainloader)

        del t_pred, label, input_x
   
        pr_net.eval()
        for input_x, label in testloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
            with torch.no_grad():
                    for i in net:
                        i.eval()
                        result_net_list.append(i(input_x))
            result_net_list = torch.cat(result_net_list)
            result_mul_add = []
            for i in range(4,result_net_list.shape[0]+1,4):
                result_mul_add.append(torch.sum(result_net_list[:i],dim = 0,keepdim= True))
            result_mul_add = torch.cat(result_mul_add)

#             optimizer.zero_grad()
            t_pred = pr_net(input_x)
            result_add = []    
            for i,j in zip(t_pred,result_mul_add):
                result_temp = []
                for m in i:
                    result_temp.append(m * result_mul_add)
                result_temp = torch.cat(result_temp)
                result_add.append(torch.sum(result_temp,dim=0,keepdim = True))
            result_add = torch.cat(result_add)
            result_add = torch.nn.functional.normalize(result_add,dim = 1)
            loss = torch.mean((result_add - label) ** 2)
            test_loss += loss.item()
        numte = len(testloader)
            
        del t_pred, label, input_x
        
        train_loss = train_loss / numtr
        test_loss = test_loss / numte
        
        time_end = time.time()
        cost = time_end - time_start
        showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss, cost)
        
        print(showStrings, end='\n')
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        
        file = open(ModuleFolder +'/' +'NET_E' + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.close()
        
        if test_loss < min_testloss:
            min_testloss = test_loss
            torch.save(pr_net.state_dict(), ModuleFolder + '/' + 'NET_E' +'-Epk{}.pth'.format(epoch+1))


In [ ]:
train_pr(200,pattern_net,trainloader,testloader,net_A,net_B,net_C,net_D)

In [ ]:
LocPatchList = ['A']
LocationList = list(range(2,5)) + list(range(22,26)) + list(range(43,49)) + list(range(64,70))
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
one_trainset    = one_point_dataset(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
trainloader=DataLoader(one_trainset, batch_size=64, shuffle=False, num_workers=8)
testloader=DataLoader(one_testset, batch_size=64, shuffle=False, num_workers=8)
# testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)

In [ ]:
class Pattern_net(nn.Module):

    def __init__(self):
        super(Pattern_net, self).__init__()
        self.upscale1 = nn.Conv2d(2, 8, kernel_size=3, padding=1)
        self.conv1 = nn.Conv2d(8, 8, kernel_size=3, padding=1)
        self.upscale2 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.fc = nn.Linear(88,88)
        self.fc2 = nn.Linear(88,88)
    def forward(self,x):
        x = self.upscale1(x)
        x = self.conv1(x)
        x = self.upscale2(x)
        x = x.view(x.shape[0], x.shape[1]* x.shape[2]*x.shape[3])
        x = self.fc(x)
        x = self.fc2(x)
        x = x.view(x.shape[0], 2, 22, 2)
        return x

In [ ]:
class Pattern_net_back(nn.Module):

    def __init__(self,input_size = 8):
        super(Pattern_net_back, self).__init__()
        self.sf = nn.Conv2d(input_size, 32, kernel_size=(3,1), padding=(1,0))
        # 64*32*22*1
        self.rgs = nn.Sequential(*[RG(32, 6, 8) for _ in range(3)])
        # 64*32*22*1
        self.conv1 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.upscale1 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.upscale2 = nn.Conv2d(16, 4, kernel_size=3, padding=1)
        self.upscale3 = nn.Conv2d(4, 2, kernel_size=3, padding=1)
#         self.fc = nn.Linear(23800,23800)
#         self.fc2 = nn.Linear(23800,23800)
    def forward(self,x):
        x = self.sf(x)
        x = self.rgs(x)
        x = self.conv1(x)
        x = self.upscale1(x)
        x = self.upscale2(x)
        x = self.upscale3(x)
#         x = x.view(x.shape[0], x.shape[1]* x.shape[2]*x.shape[3])
# #         x = self.fc(x)
# #         x = self.fc2(x)
#         print(x.shape)
#         x = x.view(x.shape[0], 2, 238, 50)
        return x

In [ ]:
class Pattern_net_back(nn.Module):

    def __init__(self,input_size = 8):
        super(Pattern_net_back, self).__init__()
        self.sf = nn.Conv2d(input_size, 32, kernel_size=3, padding=1)
        # 64*32*22*1
#         self.rgs = nn.Sequential(*[RG(32, 6, 8) for _ in range(3)])
        # 64*32*22*1
        self.conv1 = nn.Conv2d(32, 32, kernel_size=(4,1), padding=1)
        self.upscale1 = nn.Conv2d(32, 16, kernel_size=(4,1), padding=1)
        self.upscale2 = nn.Conv2d(16, 2, kernel_size=(4,4), padding=1)
        self.shapechange = nn.Conv2d(2, 2, kernel_size=(2,6), padding=(2,1))
#         self.upscale3 = nn.Conv2d(4, 2, kernel_size=(4,4), padding=1)
#         self.downscale = nn.Conv2d(2,2,kernel_size=(1,1), padding=(2,2))
#         self.fc = nn.Linear(23800,23800)
#         self.fc2 = nn.Linear(23800,23800)
    def forward(self,x):
        x = self.sf(x)
#         x = self.rgs(x)
        x = self.conv1(x)
        x = self.upscale1(x)
        x = self.upscale2(x)
        x = self.shapechange(x)
#         x = self.upscale3(x)
#         x= self.downscale(x)
#         x = x.view(x.shape[0], x.shape[1]* x.shape[2]*x.shape[3])
# #         x = self.fc(x)
# #         x = self.fc2(x)
#         print(x.shape)
#         x = x.view(x.shape[0], 2, 238, 50)
        return x

In [ ]:
pr_net(torch.rand(64,8,239,50).cuda()).shape

torch.Size([64, 2, 239, 50])

In [ ]:
def train_pr_front(Epoch,pr_net,train_loader,test_loader,*net):
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in pr_net.parameters()))
    print(NetworkParamNum)
#     model_list = [i for i in net]
    lr = 0.001
    optimizer = optim.Adam(pr_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    min_testloss = 10000
    
    testError  = []
    trainError = []
    costTime   = []
    storeStepResult = True
    net_len = len(net)
    
    for epoch in range(Epoch):
        pr_net.train()
        train_loss = 0.0
        test_loss = 0.0
        time_start = time.time()
        for input_x, label in trainloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
                        
            optimizer.zero_grad()
            t_pred = pr_net(input_x)
            t_pred = t_pred*input_x
#             with torch.no_grad():
            for n,i in enumerate(net):
                    i.eval()
                    result_net_list.append(i(t_pred[:,:,:,n].unsqueeze(dim = 3)))
            result_net_list = torch.cat(result_net_list)
            num_batch = int(input_x.shape[0])
            
            result_add = torch.zeros(num_batch,2,238,50).cuda()
            for i in range(num_batch,num_batch*net_len,num_batch):
                result_add += result_net_list[i-num_batch:i]
            result_add = result_add/net_len
#             result_add = torch.nn.functional.normalize(result_add,dim = 1)
            loss = torch.mean((result_add - label) ** 2)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
        scheduler.step()
        numtr = len(trainloader)

        del t_pred, label, input_x
   
        pr_net.eval()
        for input_x, label in testloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
            t_pred = pr_net(input_x)
            t_pred = t_pred*input_x
#             with torch.no_grad():
            for n,i in enumerate(net):
                        i.eval()
                        result_net_list.append(i(t_pred[:,:,:,n].unsqueeze(dim = 3)))
            result_net_list = torch.cat(result_net_list)
            num_batch = int(input_x.shape[0])
            result_add = torch.zeros(num_batch,2,238,50).cuda()
            for i in range(num_batch,num_batch*net_len,num_batch):
                result_add += result_net_list[i-num_batch:i]
#             result_add = torch.cat(result_add)
            result_add = result_add/net_len
#             result_add = torch.nn.functional.normalize(result_add,dim = 1)
            loss = torch.mean((result_add - label) ** 2)
            test_loss += loss.item()
        numte = len(testloader)
            
        del t_pred, label, input_x
        
        train_loss = train_loss / numtr
        test_loss = test_loss / numte
        
        time_end = time.time()
        cost = time_end - time_start
        showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss, cost)
        
        print(showStrings, end='\n')
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        file = open(ModuleFolder +'/' +Which_new + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.close()
        
        if test_loss < min_testloss:
            min_testloss = test_loss
            torch.save(pr_net.state_dict(), ModuleFolder + '/' + 'NET_A_rest_point' +'-Epk{}.pth'.format(epoch+1))


In [ ]:
def train_pr_back(Epoch,pr_net,train_loader,test_loader,*net):
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in pr_net.parameters()))
    print(NetworkParamNum)
#     model_list = [i for i in net]
    lr = 0.001
    optimizer = optim.Adam(pr_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    min_testloss = 10000
    
    testError  = []
    trainError = []
    costTime   = []
    storeStepResult = True
    net_len = len(net)
    

    for epoch in range(Epoch):
        pr_net.train()
        train_loss = 0.0
        test_loss = 0.0
        time_start = time.time()
        for input_x, label in trainloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
                        
            optimizer.zero_grad()
#             with torch.no_grad():
            for n,i in enumerate(net):
                    i.eval()
                    result_net_list.append(i(input_x))
            result_net_list = torch.cat((result_net_list[0],result_net_list[1]),1)
            num_batch = int(input_x.shape[0])
            t_pred = pr_net(result_net_list)
#             result_add = torch.nn.functional.normalize(result_add,dim = 1)
            loss = torch.mean((t_pred - label) ** 2)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
        scheduler.step()
        numtr = len(trainloader)

        del t_pred, label, input_x
   
        pr_net.eval()
        for input_x, label in testloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
            for n,i in enumerate(net):
                    i.eval()
                    result_net_list.append(i(input_x))
            result_net_list = torch.cat((result_net_list[0],result_net_list[1]),1)
            num_batch = int(input_x.shape[0])
            t_pred = pr_net(result_net_list)
#             result_add = torch.nn.functional.normalize(result_add,dim = 1)
            loss = torch.mean((t_pred - label) ** 2)
            test_loss += loss.item()
        numte = len(testloader)
            
        del t_pred, label, input_x
        
        train_loss = train_loss / numtr
        test_loss = test_loss / numte
        
        time_end = time.time()
        cost = time_end - time_start
        showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss, cost)
        
        print(showStrings, end='\n')
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        file = open(ModuleFolder +'/' +Which_new + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.close()
        
        if test_loss < min_testloss:
            min_testloss = test_loss
            torch.save(pr_net.state_dict(), ModuleFolder + '/' + 'NET_A_rest_point' +'-Epk{}.pth'.format(epoch+1))


In [ ]:
import re
class one_point_dataset_loc(Dataset):
    def __init__(self, input_path, target_path, mode = 'train', seedNN = 0):
        super(one_point_dataset_loc, self).__init__()
        self.input_data = []
        self.target_data = []
        self.loc_index = []
        for i_path in input_path:
            i_data = scio.loadmat(i_path)['tmpData']
            self.input_data.append(i_data)
            temp_list = []
            for i in range(i_data.shape[0]):
                temp_list.append(int(re.findall(r"\d+\.?\d*",i_path.split('_')[9])[0]))
            self.loc_index.append(temp_list)
        for t_path in target_path:
            self.target_data.append(scio.loadmat(t_path)['tmpData'])
        self.input_data = np.concatenate(self.input_data, axis=0)
        self.target_data = np.concatenate(self.target_data, axis=0)
        self.loc_index = np.concatenate(self.loc_index, axis=0)
        self.length = self.target_data.shape[0]
        # 随机打乱顺序
        random.seed(seedNN)
        ind_shuffle = random.sample(range(0, self.length), self.length)

        if mode == 'train':
            self.target_data = self.target_data[ind_shuffle]
            self.input_data = self.input_data[ind_shuffle]
            self.loc_index = self.loc_index[ind_shuffle]
#                 self.target_data = np.expand_dims(self.target_data, axis=0)
#                 self.input_data = np.expand_dims(self.input_data, axis=0)
        else:
            self.target_data = self.target_data[ind_shuffle]
            self.input_data = self.input_data[ind_shuffle]
            self.loc_index = self.loc_index[ind_shuffle]
        self.length = self.target_data.shape[0]
#         print(self.input_data.shape)
#         print(self.target_data.shape)
        
    def __getitem__(self, index):
        input_data = torch.from_numpy(self.input_data[index])[:, :, 0].unsqueeze(2)
        target_data = torch.from_numpy(self.target_data[index])
        location = torch.tensor([self.loc_index[index]])
        return input_data, target_data,location
        
    def __len__(self):
        return self.length

In [ ]:
LocPatchList = ['A']
LocationList = list(range(2,10)) + list(range(22,31)) + list(range(43,52)) + list(range(64,73))+ list(range(85,94))+ list(range(106,115))+ list(range(127,136))+ list(range(148,157))+ list(range(169,178))+ list(range(191,199))
VelocityList = range(1, 2)
train_loader_temp = []
train_loader_temp_l = []
test_loader_temp = []
test_loader_temp_l = []
for LocPathIndex in LocPatchList:
    train_path_d,train_path_l,val_path_d,val_path_l = DataSplitter(LocPathIndex, LocationList, VelocityList)
    train_loader_temp += train_path_d
    train_loader_temp_l += train_path_l
    test_loader_temp += val_path_d
    test_loader_temp_l+=val_path_l
one_trainset    = one_point_dataset_loc(train_loader_temp, train_loader_temp_l, mode='train', seedNN = 0)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

one_testset    = one_point_dataset_loc(test_loader_temp, test_loader_temp_l, mode='test', seedNN = 0)
trainloader=DataLoader(one_trainset, batch_size=64, shuffle=False, num_workers=8)
testloader=DataLoader(one_testset, batch_size=64, shuffle=False, num_workers=8)
# testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=8)

/datadisk/sdc/wangruibo/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def create_matrix(size = 21):
    location_matirx = torch.rand(size,size,2)
    for i in range(size):
        for j in range(size):
            location_matirx[i][j] = torch.tensor([(i+1)/size,(j+1)/size])
    location_matirx = location_matirx.reshape(location_matirx.shape[0]*location_matirx.shape[1],location_matirx.shape[2])
    return location_matirx.cuda()
# def create_pattern(input_thing,loc_index,location_matirx,size):
#     y = torch.tensor([]).cuda()
#     for i in loc_index:
#         y_temp= location_matirx[i].unsqueeze(dim = 1).repeat(1,50)
#         y = torch.cat((y,y_temp)) 
#     y_0 = y[0].unsqueeze(dim = 0)
#     y_1 = y[1].unsqueeze(dim = 0)
#     y_0 = y_0.unsqueeze(dim=0).repeat(1,1,1).unsqueeze(dim=0)
#     y_1 = y_1.unsqueeze(dim=0).repeat(1,1,1).unsqueeze(dim=0)
#     empty_0 = torch.tensor([]).cuda()
#     for i in range(size):
#         empty_0 = torch.cat((empty_0,y_0))
#     empty_1 = torch.tensor([]).cuda()
#     for i in range(size):
#         empty_1 = torch.cat((empty_1,y_1))
#     x_0 = torch.cat((input_thing[:,0].unsqueeze(dim=1),empty_0),dim = 2)
#     x_1 = torch.cat((input_thing[:,1].unsqueeze(dim=1),empty_1),dim = 2)
#     return torch.cat((x_0,x_1),dim = 1)
def create_pattern(input_thing,loc_index,location_matirx,size):
    y = torch.tensor([]).cuda()
    for i in loc_index:
        y_temp= location_matirx[i].unsqueeze(dim = 1).repeat(1,50)
        y = torch.cat((y,y_temp))
    result = torch.tensor([]).cuda()
    index = 0
    for i in range(size):
            
            y_0 = y[index].unsqueeze(dim = 0)
            y_1 = y[index+1].unsqueeze(dim = 0)
            y_0 = y_0.unsqueeze(dim=0).repeat(1,1,1).unsqueeze(dim=0)
            y_1 = y_1.unsqueeze(dim=0).repeat(1,1,1).unsqueeze(dim=0)
            x_0 = torch.cat((input_thing[i,0].unsqueeze(dim=0).unsqueeze(dim=0),y_0),dim = 2)
            x_1 = torch.cat((input_thing[i,1].unsqueeze(dim=0).unsqueeze(dim=0),y_1),dim = 2)
            result = torch.cat((torch.cat((x_0,x_1),dim = 1),result))
            index += 1
    return result
def train_pr_back_loc(Epoch,pr_net,train_loader,test_loader,*net):
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in pr_net.parameters()))
    print(NetworkParamNum)
#     model_list = [i for i in net]
    lr = 0.001
    optimizer = optim.Adam(pr_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    min_testloss = 10000
    
    testError  = []
    trainError = []
    costTime   = []
    storeStepResult = True
    net_len = len(net)
    location_matirx = create_matrix()
    for epoch in range(Epoch):
        pr_net.train()
        train_loss = 0.0
        test_loss = 0.0
        time_start = time.time()
        
        for input_x, label,loc in trainloader:
            loc = loc.squeeze()
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
                        
            optimizer.zero_grad()
            for n,i in enumerate(net):
                    i.eval()
                    result_net_list.append(i(input_x))
            result_net_list = torch.cat((result_net_list[0],result_net_list[1],result_net_list[2],result_net_list[3]),1)
            
            num_batch = int(input_x.shape[0])
            k = 2
            first_time = True
            result_net_input = torch.tensor([]).cuda()
            for i in range(4):
                result_net_list = result_net_list[:,:k]
                result_net_into = create_pattern(result_net_list,loc,location_matirx,num_batch)
                result_net_input = torch.cat((result_net_input,result_net_into),dim=1)
                k = k*2

            label_out = create_pattern(label,loc,location_matirx,num_batch)
            t_pred = pr_net(result_net_input)
            loss = torch.mean((t_pred - label_out) ** 2)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
        scheduler.step()
        numtr = len(trainloader)

        del t_pred, label, input_x
#########################################################################   
        pr_net.eval()
#         loc_predict = torch.tensor([])
#         loc_label = torch.tensor([])
        for input_x, label,loc in testloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            loc = loc.squeeze()
            result_net_list = []
            for n,i in enumerate(net):
                    i.eval()
                    result_net_list.append(i(input_x))
            result_net_list = torch.cat((result_net_list[0],result_net_list[1],result_net_list[2],result_net_list[3]),1)
            
            num_batch = int(input_x.shape[0])
            k = 2
            first_time = True
            result_net_input = torch.tensor([]).cuda()
            for i in range(4):
                result_net_list = result_net_list[:,:k]
                result_net_into = create_pattern(result_net_list,loc,location_matirx,num_batch)
                result_net_input = torch.cat((result_net_input,result_net_into),dim=1)
                k = k*2

            label_out = create_pattern(label,loc,location_matirx,num_batch)
            t_pred = pr_net(result_net_input)
#             result_add = torch.nn.functional.normalize(result_add,dim = 1)
            loss = torch.mean((t_pred - label_out) ** 2)
            test_loss += loss.item()
            t_pred = t_pred.cpu()
            label_out = label_out.cpu()
#             for i in range(num_batch):
#                 loc_predict = torch.cat((loc_predict,torch.stack((t_pred[:,:,-1,:][i][0],t_pred[:,:,-1,:][i][1]),dim = 1)))
#             for i in range(num_batch):
#                 loc_label = torch.cat((loc_label,torch.stack((label_out[:,:,-1,:][i][0],label_out[:,:,-1,:][i][1]),dim = 1)))
        numte = len(testloader)
            
        del t_pred, label, input_x
##########################################################################        
        train_loss = train_loss / numtr
        test_loss = test_loss / numte
        
        time_end = time.time()
        cost = time_end - time_start
        showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss, cost)
        
        print(showStrings, end='\n')
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        file = open(ModuleFolder +'/' +Which_new + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.close()
        
#         file = open(ModuleFolder + '/predict_loc.txt' , 'a')
#         file.write('location_predict-epk{} = '.format(epoch) + str(loc_predict.numpy().tolist()) + "\n")
#         file.close()
#         file = open(ModuleFolder + '/label_loc.txt' , 'a')
#         file.write('location_predict-epk{} = '.format(epoch) + str(loc_label.numpy().tolist()) + "\n")
#         file.close()

#         if test_loss < min_testloss:
#             min_testloss = test_loss
        torch.save(pr_net.state_dict(), ModuleFolder + '/' + 'NET_A_rest_point' +'-Epk{}.pth'.format(epoch+1))


In [ ]:
Which_new = 'NET_A_rest_point_back_100_loc'
ModuleFolder       = "/datadisk/sdc/wangruibo/Location_new_way/" + Which_new

In [ ]:
train_pr_back_loc(200,pr_net.cuda(),trainloader,testloader,net_A,net_B,net_C,net_D)

In [ ]:
class Pattern_net_back(nn.Module):

    def __init__(self,input_size = 8):
        super(Pattern_net_back, self).__init__()
        self.sf = nn.Conv2d(input_size, 32, kernel_size=(3,1), padding=(1,0))
        # 64*32*22*1
        self.rgs = nn.Sequential(*[RG(32, 6, 8) for _ in range(3)])
        # 64*32*22*1
        self.conv1 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.upscale1 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.upscale2 = nn.Conv2d(16, 4, kernel_size=3, padding=1)
        self.upscale3 = nn.Conv2d(4, 2, kernel_size=3, padding=1)
#         self.fc = nn.Linear(23800,23800)
#         self.fc2 = nn.Linear(23800,23800)
    def forward(self,x):
        x = self.sf(x)
        x = self.rgs(x)
        x = self.conv1(x)
        x = self.upscale1(x)
        x = self.upscale2(x)
        x = self.upscale3(x)
#         x = x.view(x.shape[0], x.shape[1]* x.shape[2]*x.shape[3])
# #         x = self.fc(x)
# #         x = self.fc2(x)
#         print(x.shape)
#         x = x.view(x.shape[0], 2, 238, 50)
        return x
pr_net = Pattern_net_back().cuda()

In [ ]:
class loc_net(nn.Module):

    def __init__(self):
        super(loc_net, self).__init__()
        self.fc = nn.Linear(23800,2)
        self.fc2 = nn.Linear(2,2)
    def forward(self,x):
        x = x.view(x.shape[0],-1)
        x = self.fc(x)
        x = self.fc2(x)
        return x

In [ ]:
location_net = loc_net().cuda()

In [ ]:
def draw_point(predict_loc,true_loc):
    plt.figure(figsize=(6, 6))
    plt.xlim(0,1)
    plt.ylim(0,1)
    predict_loc = predict_loc.cpu().detach().numpy()
    true_loc = true_loc.cpu().detach().numpy()
    
    for p,t in zip(predict_loc,true_loc):
        colors = np.random.choice(list(mcolors.CSS4_COLORS.keys()),replace=False)
        plt.scatter(p[0],p[1],marker = 'x',color = colors)
        plt.scatter(t[0],t[1],marker = 'o',color = colors)
        plt.legend(("Predict Location","True Location"))
    plt.show()

In [ ]:
def create_matrix(size = 21):
    location_matirx = torch.rand(size,size,2)
    for i in range(size):
        for j in range(size):
            location_matirx[i][j] = torch.tensor([(i+1)/size,(j+1)/size])
    location_matirx = location_matirx.reshape(location_matirx.shape[0]*location_matirx.shape[1],location_matirx.shape[2])
    return location_matirx.cuda()


def train_pr_back_loc_new_way(Epoch,pr_net,train_loader,test_loader,*net):
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in pr_net.parameters()))
    print(NetworkParamNum)
    NetworkParamNum_linear = "Total number of paramerters in networks of linear is {}  ".format(sum(x.numel() for x in location_net.parameters()))
    print(NetworkParamNum_linear)
#     model_list = [i for i in net]
    lr = 0.001
    optimizer = optim.Adam(pr_net.parameters(), lr=lr)
    optimizer_loc = optim.Adam(pr_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    scheduler_loc = optim.lr_scheduler.StepLR(optimizer_loc, step_size=10, gamma=0.9)
    min_testloss = 10000
    
    testError  = []
    trainError = []
    costTime   = []
    storeStepResult = True
    net_len = len(net)
    location_matirx = create_matrix()
    for epoch in range(Epoch):
        pr_net.train()
        location_net.train()
        train_loss = 0.0
        test_loss = 0.0
        time_start = time.time()
        train_loss_loc = 0.0
        test_loss_loc = 0.0
        for input_x, label,loc in trainloader:
            loc = loc.squeeze()
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            result_net_list = []
                        
            optimizer.zero_grad()
            for n,i in enumerate(net):
                    i.eval()
                    result_net_list.append(i(input_x))
            result_net_list = torch.cat((result_net_list[0],result_net_list[1],result_net_list[2],result_net_list[3]),1)
            
            num_batch = int(input_x.shape[0])
            

            t_pred = pr_net(result_net_list)
            loss = torch.mean((t_pred - label) ** 2)
            loss.backward(retain_graph = True)
            optimizer.step()
            train_loss += float(loss.item())
            train_step = True
            if train_step:
                pr_net.eval()
                optimizer_loc.zero_grad()
                t_pred_out = pr_net(result_net_list)
                loc_point = location_net(t_pred_out)
                loc_label = location_matirx[loc]
                loss_loc = torch.mean((loc_point - loc_label) ** 2)
                loss_loc.backward()
                optimizer_loc.step()
                train_loss_loc += float(loss_loc.item())
            
        scheduler.step()
        scheduler_loc.step()
        numtr = len(trainloader)

        del t_pred, label, input_x
#########################################################################   
        pr_net.eval()
        location_net.eval()
        for input_x, label,loc in testloader:
            input_x = input_x.cuda().float()
            label = label.cuda().float()
            loc = loc.squeeze()
            result_net_list = []
            for n,i in enumerate(net):
                    i.eval()
                    result_net_list.append(i(input_x))
            result_net_list = torch.cat((result_net_list[0],result_net_list[1],result_net_list[2],result_net_list[3]),1)
            
            num_batch = int(input_x.shape[0])
            t_pred = pr_net(result_net_list)
            loss = torch.mean((t_pred - label) ** 2)
            test_loss += loss.item()
            eval_step = True
            if eval_step:
                
                t_pred_out = pr_net(result_net_list)
                loc_point = location_net(t_pred_out)
                loc_label = location_matirx[loc]
                loss_loc = torch.mean((loc_point - loc_label) ** 2)
                test_loss_loc += float(loss_loc.item())
        numte = len(testloader)
            
        del t_pred, label, input_x
##########################################################################        
        train_loss = train_loss / numtr
        test_loss = test_loss / numte
        train_loss_loc = train_loss_loc / numtr
        test_loss_loc = test_loss_loc / numte
        time_end = time.time()
        cost = time_end - time_start
        showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][LocTraingLoss: %3.5f][LocTestingLoss:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss,train_loss_loc,test_loss_loc, cost)
        
        print(showStrings, end='\n')
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        file = open(ModuleFolder +'/' +Which_new + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('TrainingLoss_loc = ' + str(train_loss_loc) + "\n")
        file.write('TestingLoss_loc = '  + str(test_loss_loc) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.close()
        
#         file = open(ModuleFolder + '/predict_loc.txt' , 'a')
#         file.write('location_predict-epk{} = '.format(epoch) + str(loc_predict.numpy().tolist()) + "\n")
#         file.close()
#         file = open(ModuleFolder + '/label_loc.txt' , 'a')
#         file.write('location_predict-epk{} = '.format(epoch) + str(loc_label.numpy().tolist()) + "\n")
#         file.close()

#         if test_loss < min_testloss:
#             min_testloss = test_loss
        torch.save(pr_net.state_dict(), ModuleFolder + '/' + 'NET_A_rest_point' +'-Epk{}.pth'.format(epoch+1))
        torch.save(location_net.state_dict(), ModuleFolder + '/' + 'NET_A_rest_point_loc' +'-Epk{}.pth'.format(epoch+1))

In [ ]:
Which_new = 'NET_A_rest_point_back_100_loc_linear'
ModuleFolder       = "/datadisk/sdc/wangruibo/Location_new_way/" + Which_new

In [ ]:
train_pr_back_loc_new_way(200,pr_net.cuda(),trainloader,testloader,net_A,net_B,net_C,net_D)

Total number of paramerters in networks is 141638  
Total number of paramerters in networks of linear is 47608  
[Epoch:   1/200][TraingLoss: 0.18996][TestingLoss:0.12312][LocTraingLoss: 0.02512][LocTestingLoss:0.06061][CostTime: 170.11816 s]
[Epoch:   2/200][TraingLoss: 0.10497][TestingLoss:0.09405][LocTraingLoss: 0.01037][LocTestingLoss:0.03783][CostTime: 169.70978 s]
[Epoch:   3/200][TraingLoss: 0.07808][TestingLoss:0.07053][LocTraingLoss: 0.00638][LocTestingLoss:0.01937][CostTime: 170.08244 s]
[Epoch:   4/200][TraingLoss: 0.06325][TestingLoss:0.05917][LocTraingLoss: 0.00506][LocTestingLoss:0.01699][CostTime: 169.52115 s]


In [ ]:
def draw_plot(data,name):
    f = plt.figure(figsize=(20,12))
    ax = f.add_subplot(121)
    ax2 = f.add_subplot(122)
    ax.imshow(data.squeeze().cpu().detach().numpy()[0])
    ax2.imshow(data.squeeze().cpu().detach().numpy()[1])
    ax.set_title('Real part in '+name)
    ax.set_xlabel('Time')
    ax.set_xlabel('Frequency')
    for i in range(0,238,11):
        rect = patches.Rectangle((0, i), 2, 2, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    for i in range(0,238,11):
        rect = patches.Rectangle((0, i), 2, 2, linewidth=1, edgecolor='r', facecolor='none')
        ax2.add_patch(rect)
    ax2.set_xlabel('Time')
    ax2.set_xlabel('Frequency')
    ax2.set_title('Image part in '+name)
    plt.savefig(name + '.jpg')

    plt.show()

In [ ]:
# For testing of basic
# ModuleList  = [178]
# ModelFolder = ModuleFolder + "Checkpoints3x3/"
net_list = [net_A,net_B,net_C,net_D]
LocationList = range(1, 1682)
VelocityList = range(1,2)
ModuleList = 153
# ResultFolder  = ModuleFolder + 'NetworkRes3x3/'
num_batch = 85
ModelName = '/datadisk/sdc/wangruibo/Location_new_way/NET_E/NET_E-Epk{}.pth'.format(ModuleList)
net = pr_net.cuda()
net.load_state_dict(torch.load(ModelName))
#     ModelName   = ModelFolder + ThisFileName + '-Epk{}.pth'.format(Model)
#     print(ModelName)
for LocDiv in LocationList:
    for VelDiv in VelocityList:
        print('Process Location {} with Epoch {}'.format(LocDiv, ModuleList))
        TestingDataFolder  = "/datadisk/sdc/wangruibo/share1/QWL_DataBase/SuperRes_ChanEst/SISO_1x1/Diversity_LocationVelocity/210421_TestData_NLOS1Square/"
        TestingDataName   = TestingDataFolder + 'Loc E/RawCe_PilotOnly/10ReDmrs_Ant1_RxPos{}_RxVel{}_ModOrd1_RawCe_PilotOnly.mat'.format(LocDiv, VelDiv)
        
        net.eval()
        
        RawCE_Pilot = scio.loadmat(TestingDataName)['tmpData']

        RawCE_Pilot = torch.from_numpy(RawCE_Pilot[:,:,:,0]).unsqueeze(3)
        RawCE_Pilot = RawCE_Pilot.float().cuda()
#         output_data = net(RawCE_Pilot).detach().cpu().numpy()
        output_data = net(RawCE_Pilot)
        result_net_list = []
        for n,i in enumerate(net_list):
            i.eval()
            result_net_list.append(i(output_data[:,:,:,n].unsqueeze(dim = 3)))
        result_net_list = torch.cat(result_net_list)
        result_add = torch.zeros(num_batch,2,238,50).cuda()
        for i in range(num_batch,num_batch*4,num_batch):
                result_add += result_net_list[i-num_batch:i]
        result_add = result_add.detach().cpu().numpy()
        savePath    = '/datadisk/sdc/wangruibo/Location_new_way/MAT_E/NEW-PR-Epk{}-Loc{}-Vel{}.mat'.format(ModuleList, LocDiv, VelDiv)
        scio.savemat(savePath, {'tmpData': result_add})

Process Location 1 with Epoch 153
Process Location 2 with Epoch 153
Process Location 3 with Epoch 153
Process Location 4 with Epoch 153
Process Location 5 with Epoch 153
Process Location 6 with Epoch 153
Process Location 7 with Epoch 153
Process Location 8 with Epoch 153
Process Location 9 with Epoch 153
Process Location 10 with Epoch 153
Process Location 11 with Epoch 153
Process Location 12 with Epoch 153
Process Location 13 with Epoch 153
Process Location 14 with Epoch 153
Process Location 15 with Epoch 153
Process Location 16 with Epoch 153
Process Location 17 with Epoch 153
Process Location 18 with Epoch 153
Process Location 19 with Epoch 153
Process Location 20 with Epoch 153
Process Location 21 with Epoch 153
Process Location 22 with Epoch 153
Process Location 23 with Epoch 153
Process Location 24 with Epoch 153
Process Location 25 with Epoch 153
Process Location 26 with Epoch 153
Process Location 27 with Epoch 153
Process Location 28 with Epoch 153
Process Location 29 with Epoc